# Extração de tweets para uma base de dados local sqllite

Para correr este notebook é preciso ter uma conta de developer ou de Aademic Researcher no Twitter

In [ ]:
import requests
import os
import json

bearer_token = '<copiar token da conta do twitter>'
search_url = "https://api.twitter.com/2/tweets/search/recent"

# Optional params: start_time,end_time,since_id,until_id,max_results,next_token,
# expansions,tweet.fields,media.fields,poll.fields,place.fields,user.fields
query_params = {'query': '#trigo','tweet.fields': 'author_id,created_at,text'}

def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """
    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2FullArchiveSearchPython"
    return r

def connect_to_endpoint(url, params):
    response = requests.request("GET", search_url, auth=bearer_oauth, params=query_params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

json_response = connect_to_endpoint(search_url, query_params)
#Caso queiram confirmar que está tudo a funcionar
#print(json.dumps(json_response, indent=4, sort_keys=True))


200
INSERT INTO TWEET (id,author_id,created_at,text) VALUES ('1540943171399589890','47302521','2022-06-26T06:20:56.000Z','Rusia impide la salida de 25 millones de toneladas de #trigo ante la indignación de los expertos, que temen una crisis alimentaria global  https://t.co/OnnkOdyJja')
INSERT INTO TWEET (id,author_id,created_at,text) VALUES ('1540891517371535360','919011836','2022-06-26T02:55:40.000Z','Desarrollan una variedad de #trigo más nutritivo y resistente Vía @Munagropecuario https://t.co/yJNfmZt9M6 #mejoramientogenético #nutrición')
INSERT INTO TWEET (id,author_id,created_at,text) VALUES ('1540880781358338048','3227065697','2022-06-26T02:13:01.000Z','RT @CristianCZajac: #Sabado de datos #heladas #nevadas #trigo #Junio2022 #Pronosticos #forecast #Lluvias #Tormentas #suelos #BahiaBlanca #B…')
INSERT INTO TWEET (id,author_id,created_at,text) VALUES ('1540879151758020609','3227065697','2022-06-26T02:06:32.000Z','RT @CristianCZajac: #Sabado de estáticos #heladas #nevadas #trigo #Ju

In [ ]:
#Colocar numa base de dados sqllite
import sqlite3
#Esta bd é local, para por no google drive é preciso montar a drive
conn = sqlite3.connect('twitter2.db')
c = conn.cursor()
#c.execute("DROP TABLE tweet")
#c.execute("CREATE TABLE tweet (id VARCHAR, author_id VARCHAR, created_at DATETIME, text VARCHAR)")

for linha in json_response['data']:
  #print ('INSERT INTO TWEET (id,author_id,created_at,text) VALUES (\''+ linha['id'] + '\',\'' + linha['author_id'] + '\',\''+ linha['created_at'] + '\',\''+ linha['text'] +'\')')
  c.execute('INSERT INTO TWEET (id,author_id,created_at,text) VALUES (\''+ linha['id'] + '\',\'' + linha['author_id'] + '\',\''+ linha['created_at'] + '\',\''+ linha['text'] +'\')')
c.close()


In [ ]:
#Tirar da bd
for linha in c.execute('select * from tweet'):
  print(linha)

In [ ]:
#Tirar da bd e colocar num pandas dataframe
import pandas as pd
df = pd.read_sql_query("select * from tweet", conn)
df.head(10)

,id,author_id,created_at,text
0,1540943171399589890,47302521,2022-06-26T06:20:56.000Z,Rusia impide la salida de 25 millones de tonel...
1,1540891517371535360,919011836,2022-06-26T02:55:40.000Z,Desarrollan una variedad de #trigo más nutriti...
2,1540880781358338048,3227065697,2022-06-26T02:13:01.000Z,RT @CristianCZajac: #Sabado de datos #heladas ...
3,1540879151758020609,3227065697,2022-06-26T02:06:32.000Z,RT @CristianCZajac: #Sabado de estáticos #hela...
4,1540879122196570112,3227065697,2022-06-26T02:06:25.000Z,#Sabado de datos #heladas #nevadas #trigo #Jun...
5,1540875481930731520,143008399,2022-06-26T01:51:57.000Z,RT @NOUREEN_SHIRAZ: 🇹🇷 #Турция 🇹🇷 #Turkey Hay ...
6,1540868439782432769,3227065697,2022-06-26T01:23:58.000Z,#Sabado de estáticos #heladas #nevadas #trigo ...
7,1540860852521369603,1602371683,2022-06-26T00:53:49.000Z,@Osinttechnical Va madurando el #Trigo en #Ucr...
8,1540859116649840640,3227065697,2022-06-26T00:46:55.000Z,RT @CristianCZajac: #Sabado de #heladas #nevad...
9,1540832207488565249,140426707,2022-06-25T23:00:00.000Z,"""Cultivos argentinos"", la web que crearon inve..."


A partir daqui podem fazer o que quiserem, analisar o texto, análise de sentimentos, etc.